In [ ]:
#!/usr/bin/env python
# -*- coding: latin-1 -*-
import pandas as pd # pandas is a data manipulation library
import numpy as np #provides numerical arrays and functions to manipulate the arrays efficiently
import csv
import logging
from SPARQLWrapper import SPARQLWrapper,JSON, SPARQLWrapper2, POST,XML,GET
from collections import defaultdict
from tqdm.notebook import tqdm
import query_helper_functions as q_helper
import sparql_query as sparql_query
import imp
imp.reload(q_helper)
imp.reload(sparql_query)


In [ ]:

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
tqdm.pandas(desc="progress bar")


In [ ]:
ml_file_movies_df = pd.read_csv('../dataset/ml_movies.csv')

In [ ]:
ml_file_movies_df

In [ ]:
ml_file_movies_df[10:20]

In [ ]:
#clean titles in ml data
#delete any record with empty title
ml_file_movies_df= ml_file_movies_df.loc[~(ml_file_movies_df['title'].str=="")]
# extract the year from the title and save it in a new column
ml_file_movies_df['year'] =  ml_file_movies_df.title.str.extract(r'\(([0-9]+)\)')
# remove any parentheses and text in between from the title
ml_file_movies_df['title'] = ml_file_movies_df['title'].str.replace(r"\(.*\)","")
#remove white spaces from the title and convert it to lower case
ml_file_movies_df.title = ml_file_movies_df.title.str.strip().str.lower()
#remove double qoutation from title
ml_file_movies_df['title'] = ml_file_movies_df['title'].str.replace("\"","")
#move "the" from the end of the title to the begining
ml_file_movies_df['title'] = ml_file_movies_df['title'].apply(lambda row: 'the ' + row.replace(", the", "") if ", the" in row else row)

In [ ]:
ml_file_movies_df= ml_file_movies_df.loc[~(ml_file_movies_df['title'].isnull())]

In [ ]:
#copy thedata into new dataframe
ml_movies_df = ml_file_movies_df.copy()
ml_movies_df.describe

In [ ]:
# importing the string module
import string
# special characters
special_chars = string.punctuation
special_chars

In [ ]:
ml_movies_df[ml_movies_df.title.apply(lambda title: True if title in special_chars else False)]

In [ ]:
ml_movies_df[ml_movies_df['movieId']==115263].title

In [ ]:
ml_movies_df[ml_movies_df['title']==""]

In [ ]:
movies_list = ml_movies_df['title'].values.tolist()

In [ ]:
print("Number of movies from MovieLens dataset is {} ".format(len(movies_list)))

# Query DBPedia for MovieLens Data

In [ ]:
#uncomment either the call to query lod to refresh the data or the load from file
endpoint = "https://dbpedia.org/sparql"
return_format = XML
request_method= POST
chunck_size = 2000
#movies_info_result_df = q_helper.retrieve_data_for_movies_list(sparql_query.dbpedia_movies_info_query, movies_list, chunck_size,'string', endpoint, return_format, request_method)
movies_info_df = movies_info_result_df[0].copy()
failed_movies_info_list = movies_info_result_df[1].copy()
#save raw results of movies info query to file
movies_info_df.to_csv("generated_dataset/dbpedia_movie_info_query_raw_results.csv",index=False)

In [ ]:
movies_info_df['movie_id']= movies_info_df.pedia_movie.str.split('http://dbpedia.org/resource/').str[1]

movies_info_df = movies_info_df.fillna('NOT_FOUND')
movies_info_df['country']= movies_info_df['country'].apply(lambda country: 
                                                 country.split('http://dbpedia.org/resource/')[1] if 'dbpedia.org/resource/' in country else country )



In [ ]:
#Group movies info results to extract the directors, writers, and titles
movies_info_grouped = movies_info_df.groupby(['movie_id','title'], as_index=False)
#aggregate the directors and directors in one column for each separated by |
dbpedia_info_df = movies_info_grouped.agg({
                                        'director_name':lambda x:  '|'.join(set(x)),
                                        'writer_name':lambda x: '|'.join(set(x)),                                      
                                        'country':lambda x: '|'.join(set(x))
                                        })
#save grouped data to file
dbpedia_info_df.to_csv("generated_dataset/dbpedia_movie_info_grouped.csv", index=False)

In [ ]:
dbpedia_info_df

# Data analysis for the retrieved movies info (director, writer, country) from DBpedia

In [ ]:
pedia_info_no_features = len(dbpedia_info_df[
                         ((dbpedia_info_df.director_name == 'NOT_FOUND')
                          & (dbpedia_info_df.writer_name == 'NOT_FOUND')                          
                          & (dbpedia_info_df.country == 'NOT_FOUND'))])

 
pedia_info_all_features = len(dbpedia_info_df[
                              ((dbpedia_info_df.director_name != 'NOT_FOUND')
                              & (dbpedia_info_df.writer_name != 'NOT_FOUND')                          
                              & (dbpedia_info_df.country != 'NOT_FOUND'))])

pedia_info_no_country = len(dbpedia_info_df[
                             ((dbpedia_info_df.director_name != 'NOT_FOUND')
                          &  (dbpedia_info_df.writer_name != 'NOT_FOUND')                          
                          &  (dbpedia_info_df.country == 'NOT_FOUND'))])

pedia_info_no_writer = len(dbpedia_info_df[
                             ((dbpedia_info_df.director_name != 'NOT_FOUND')
                          &  (dbpedia_info_df.writer_name == 'NOT_FOUND')                          
                          &  (dbpedia_info_df.country != 'NOT_FOUND'))])

pedia_info_no_director = len(dbpedia_info_df[
                             ((dbpedia_info_df.director_name == 'NOT_FOUND')
                          &  (dbpedia_info_df.writer_name != 'NOT_FOUND')                          
                          &  (dbpedia_info_df.country != 'NOT_FOUND'))])


In [ ]:
print("Number of ML movies retrieved from DBpedia with their info is {}".format(len(dbpedia_info_df)))
print("Number of movies missing all features is {}".format(pedia_info_no_features))
print("Number of movies having all features is {}".format(pedia_info_all_features))
print("Number of movies missing ONLY the country feature is {}".format(pedia_info_no_country))
print("Number of movies missing ONLY the writer feature is {}".format(pedia_info_no_writer))
print("Number of movies missing ONLY the director feature is {}".format(pedia_info_no_director))

In [ ]:
# Load DBpedia movies information from last query results
#dbpedia_info_df= pd.read_csv("generated_dataset/dbpedia_movie_info_grouped.csv")
#dbpedia_info_df

# Query DBpedia for the actors of the ML movies

In [ ]:
#uncomment either the query to refresh the data from the source or load the previously retrieved data from file
endpoint = "https://dbpedia.org/sparql"
return_format = XML
request_method= POST
chunk_size = 2000
movies_actors_results_df = q_helper.retrieve_data_for_movies_list(sparql_query.dbpedia_movies_actors_query,movies_list,chunk_size,'string',endpoint, return_format,request_method)
movies_actors_df= movies_actors_results_df[0].copy()
failed_movies_actors_list = movies_actors_results_df[1].copy()
#save movies actors results to file
movies_actors_df.to_csv("generated_dataset/dbpedia_movies_actors_query_raw_results.csv",index=False)

In [ ]:
movies_actors_df

In [ ]:

#create movie id by extracting the dbpedia movie id from the link
movies_actors_df['movie_id']= movies_actors_df.pedia_movie.str.split('http://dbpedia.org/resource/').str[1]
movies_actors_df = movies_actors_df.fillna('NOT_FOUND')

In [ ]:
movies_actors_df

In [ ]:
#group the actors per movie and save data to file
movies_actors_grouped = movies_actors_df.groupby(['movie_id'], as_index=False)

#aggregate the actors in one column for each movie separated by |
dbpedia_movies_actors_df = movies_actors_grouped.agg({'actor_name':lambda x: '|'.join(set(x))
                                         })

dbpedia_movies_actors_df.to_csv("generated_dataset/dbpedia_actors_grouped.csv",index=False)


In [ ]:
dbpedia_movies_actors_df.describe

In [ ]:
dbpedia_movies_actors_df.isnull().values.any()

In [ ]:
dbpedia_movies_no_actors = len(dbpedia_movies_actors_df[(dbpedia_movies_actors_df.actor_name == 'NOT_FOUND')])


In [ ]:
print("Number of ML movies found in DBpedia while querying for actors is {}".format(len(dbpedia_movies_actors_df)))
print("Number of movies retrieved from DBpedia and missing actors is {}".format(dbpedia_movies_no_actors))

In [ ]:
# Load DBpedia movies actors from last query results
#dbpedia_movies_actors_df= pd.read_csv("generated_dataset/dbpedia_actors_grouped.csv")
#dbpedia_movies_actors_df

In [ ]:
#merge dbpedia dataframes to create a combined yago dataset with the attributes, movie_id,title,actor_name,director_name,genre,country
dbpedia_movies_df = dbpedia_info_df.merge(dbpedia_movies_actors_df, how="inner", on=['movie_id'])

dbpedia_movies_df.reset_index(drop= True)
dbpedia_movies_df

# Query YAGO for MovieLens Data

In [ ]:
def retrieve_data_for_movies_list(query, search_list, chunck_size, search_data_type, endpoint, return_format,request_method):
    
    data_chunck = [movie.strip().lower() for movie in search_list]
    final_search_list = '\"""' + '\""",\"""'.join(data_chunck) + '\"""' 
    sparql = SPARQLWrapper2("https://yago-knowledge.org/sparql/query")  
    #sparql.setReturnFormat(JSON)
    #sparql.setRequestMethod(POST)
    sparql.setQuery(query%final_search_list) 
    results = sparql.query()
   # results = execute_query_for_movies_list(query,final_search_list,endpoint, return_format,request_method)       
    print(results)

    return results
results = retrieve_data_for_movies_list(sparql_query.yago_movie_info_query, movies_list[0:5], chunck_size,'string', endpoint, return_format, request_method)

In [ ]:
def get_sparql_dataframe(result):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    processed_results = result.fullResult
    processed_results
    cols = processed_results['head']['vars']
   # print(cols)

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

#function to retrieve the actors for list of movies
def execute_query(query, endpoint, return_format):
    sparql = SPARQLWrapper2(endpoint)  
    sparql.setQuery(query) 
    sparql.setReturnFormat(return_format)   
    sparql.setMethod(POST)
  
    results = sparql.query()
    return results

def execute_query_for_movies_list(query,movies_list, endpoint,return_format,request_method):
    sparql = SPARQLWrapper2(endpoint)  
    sparql.setQuery(query%movies_list) 
   # print(sparql.queryString)
    sparql.setReturnFormat(return_format)   
    sparql.setMethod(request_method)
   # print(sparql)
    results = sparql.query()
    
    return results


In [ ]:
#uncomment either the call to query yago to refresh the data or the load from file
#endpoint = "http://yago-knowledge.org/sparql"
endpoint="https://yago-knowledge.org/sparql/query"
return_format = JSON
request_method= POST
chunck_size = 500
movies_info_result_df1 = q_helper.retrieve_data_for_movies_list(sparql_query.yago_movie_info_query, movies_list, chunck_size,'string', endpoint, return_format, request_method)
movies_info_df1 = movies_info_result_df[0].copy()
failed_movies_info_list = movies_info_result_df[1].copy()
#save raw results of movies info query to file
#movies_info_df.to_csv("generated_dataset/yago_movie_info_query_raw_results.csv",index=False)

In [ ]:
movies_info_df = movies_info_df.fillna('NOT_FOUND')

In [ ]:
movies_info_df.info()

In [ ]:
#Group movies info results to extract the directors, writers, and titles
movies_info_grouped = movies_info_df.groupby(['movie_id','title'], as_index=False)
#aggregate the directors and directors in one column for each separated by |
yago_info_df = movies_info_grouped.agg({
                                        'director_name':lambda x: np.nan if x is None else '|'.join(set(x)),
                                        'genre':lambda x: '|'.join(set(x)),                                      
                                        'country':lambda x: '|'.join(set(x))
                                        })
#save grouped data to file
yago_info_df.to_csv("generated_dataset/yago_movie_info_grouped.csv", index=False)

In [ ]:
yago_info_df.describe

In [ ]:
print("Number of ML movies retrieved from YAGO with their info is {}".format(len(yago_info_df)))

# Load yago movies information

In [ ]:
yago_info_df= pd.read_csv("generated_dataset/yago_movie_info_grouped.csv")
yago_info_df

# Query YAGO for the actors of the ML movies

In [39]:
#uncomment either the query to refresh the data from the source or load the previously retrieved data from file
#endpoint = "https://yago-knowledge.org/sparql"
endpoint="https://yago-knowledge.org/sparql/query"
return_format = JSON
request_method= POST
chunk_size = 200
#movies_actors_results_df = q_helper.retrieve_data_for_movies_list(sparql_query.yago_movies_actors_query,movies_list,chunk_size,'string',endpoint, return_format,request_method)
movies_actors_df= movies_actors_results_df[0].copy()
failed_movies_actors_list = movies_actors_results_df[1].copy()
#save raw results of movies actors query to file
movies_actors_df.to_csv("generated_dataset/yago_movie_actors_query_raw_results.csv",index=False)

In [55]:
movies_actors_df.to_csv("generated_dataset/yago_movie_actors_query_raw_results_COMPLETE.csv",index=False)

In [36]:
failed_list = [movie.replace('"""', "") for movie in failed_movies_actors_list]
len(failed_list)

In [38]:
second_movies_actors_results_df = q_helper.retrieve_data_for_movies_list(sparql_query.yago_movies_actors_query,failed_list,chunk_size,'string',endpoint, return_format,request_method)

Searching by movie title
Processing movies between 0 200

Results data frame length = 2768 

Searching by movie title
Processing movies between 200 400

Results data frame length = 5823 

Searching by movie title
Processing movies between 400 600

Results data frame length = 8400 

Searching by movie title
Processing movies between 600 800

Results data frame length = 11196 

Searching by movie title
Processing movies between 800 1000

Results data frame length = 13931 

Searching by movie title
Processing movies between 1000 1053

Results data frame length = 14626 



In [54]:
movies_actors_df

,yago_movie,title,actor_name
0,http://yago-knowledge.org/resource/Smoke_(film),Smoke,Ashley Judd
1,http://yago-knowledge.org/resource/Leaving_Las...,Leaving Las Vegas,Carey Lowell
2,http://yago-knowledge.org/resource/Leaving_Las...,Leaving Las Vegas,Danny Huston
3,http://yago-knowledge.org/resource/Happy_Gilmore,Happy Gilmore,Adam Sandler
4,http://yago-knowledge.org/resource/Happiness_I...,Happiness Is in the Field,Alexandra London
...,...,...,...
690,http://yago-knowledge.org/resource/Shock_to_th...,Shock to the System,NOT_FOUND
691,http://yago-knowledge.org/resource/A_(1965_film),A,NOT_FOUND
692,http://yago-knowledge.org/resource/Joan_of_Arc...,Joan of Arc,NOT_FOUND
693,http://yago-knowledge.org/resource/Joan_of_Arc...,Joan of Arc,NOT_FOUND


In [ ]:
#movies_actors_results_df = q_helper.retrieve_data_for_movies_list(sparql_query.yago_movies_actors_query,movies_list,chunk_size,'string',endpoint, return_format,request_method)

In [53]:
movies_actors_df = movies_actors_df.fillna('NOT_FOUND')

In [ ]:
#group the actors per movie and save data to file
movies_actors_grouped = movies_actors_df.groupby(['title'], as_index=False)

#aggregate the actors in one column for each movie separated by |
yago_movies_actors_df = movies_actors_grouped.agg({'actor_name':lambda x: '|'.join(set(x))
                                         })

yago_movies_actors_df.to_csv("generated_dataset/yago_actors_grouped.csv",index=False)


In [ ]:
yago_movies_actors_df.describe

In [ ]:
yago_movies_actors_df.isnull().values.any()

In [ ]:
print("Number of ML movies retrieved from LOD with their actors is {}".format(len(dbpedia_movies_actors_df)))

# Load movies actors from last query results

In [ ]:
yago_movies_actors_df= pd.read_csv("generated_dataset/yago_actors_grouped.csv")
yago_movies_actors_df

# Merge the actors dataframe with the movies information to create one dataframe for YAGO movies

In [ ]:
#merge yago dataframes to create a combined yago dataset with the attributes, movie_id,title,actor_name,director_name,genre,country
yago_movies_df = yago_info_df.merge(yago_movies_actors_df, how="inner", on=['movie_id'])

yago_movies_df.reset_index(drop= True)
yago_movies_df

In [ ]:
final_movies_df = yago_movies_df.merge(dbpedia_movies_df, how="outer", on=['movie_id','title'])
final_movies_df = final_movies_df.fillna('NOT_FOUND')
final_movies_df.describe

In [ ]:
to_keep_val = "NOT_FOUND"
other_val = "other_val"

def select_values(to_keep_val, other_val):
    if (str(to_keep_val) != "NOT_FOUND"):
        return to_keep_val
    else:
        return other_val
    
select_values(to_keep_val, other_val)


final_movies_df['actor_name'] = final_movies_df.progress_apply(lambda row: select_values(row.actor_name_y, row.actor_name_x), axis=1)
final_movies_df['director_name'] = final_movies_df.progress_apply(lambda row: select_values(row.director_name_x, row.director_name_y), axis=1)
final_movies_df['country'] = final_movies_df.progress_apply(lambda row: select_values(row.country_x, row.country_y), axis=1)

In [ ]:
final_movies_df[['actor_name_x','actor_name_y', 'actor_name']].head(50)

In [ ]:
final_movies_df[['director_name_x','director_name_y', 'director_name']].head(50)

In [ ]:
final_movies_df

In [ ]:
all_movies_df = final_movies_df[['movie_id','title','actor_name','director_name','writer_name','country','genre']].copy()

In [ ]:
all_movies_df

In [ ]:
#remove duplicated movie_id
all_movies_df= all_movies_df[~all_movies_df.duplicated('movie_id')]
all_movies_df

In [ ]:
all_movies_df.duplicated('movie_id').any().sum()

In [ ]:
all_movies_df = all_movies_df[~((all_movies_df.actor_name == 'NOT_FOUND')
                          & (all_movies_df.director_name == 'NOT_FOUND')
                          & (all_movies_df.writer_name == 'NOT_FOUND')
                          & (all_movies_df.genre == 'NOT_FOUND')
                          & (all_movies_df.country == 'NOT_FOUND'))]
all_movies_df

In [ ]:
complete_dataset= all_movies_df[((all_movies_df.actor_name != 'NOT_FOUND')
                          & (all_movies_df.director_name != 'NOT_FOUND')
                          & (all_movies_df.writer_name != 'NOT_FOUND')
                          & (all_movies_df.genre != 'NOT_FOUND')
                          & (all_movies_df.country != 'NOT_FOUND'))]
complete_dataset

In [ ]:
missing_only_genre_dataset= all_movies_df[((all_movies_df.actor_name == 'NOT_FOUND')
                          & (all_movies_df.director_name != 'NOT_FOUND')
                          & (all_movies_df.writer_name != 'NOT_FOUND')
                          & (all_movies_df.genre != 'NOT_FOUND')
                          & (all_movies_df.country != 'NOT_FOUND'))]
missing_only_genre_dataset

In [ ]:
all_movies_df